In [7]:
import os
import pymysql
import pandas as pd
import numpy as np
from dotenv import load_dotenv, find_dotenv

In [8]:
from customerclustering.db_connection import Db
from customerclustering.get_training_data import *
from customerclustering.baseline import *
from customerclustering.trainer_v0 import *

# Model Predict

In [28]:
import joblib
import pandas as pd

baseline = joblib.load("../models/model_v0_withoutpca.joblib")

training_data_df = pd.read_csv("../raw_data/training_data_2.csv", index_col=[0])

c = baseline.predict(training_data_df)

In [29]:
training_data_df_labels = training_data_df.copy()
training_data_df_labels['col_labels'] = c

In [3]:
individual_user = baseline.predict(training_data_df.iloc[:1, :])

In [4]:
individual_user

array([0], dtype=int32)

In [6]:
# training_data_df_labels.to_csv('../raw_data/df_with_labels2.csv')

In [9]:
cluster_df = pd.read_csv("../raw_data/df_with_labels2.csv",index_col=[0])
# cluster_df.head()

# Features Categorisation

## All features

In [11]:
import io
buffer = io.StringIO()
training_data_df.info(buf=buffer)
lines = buffer.getvalue().splitlines()
column_df = (pd.DataFrame([x.split() for x in lines[6:-2]], columns=lines[3].split())
       .drop('Count',axis=1)
       .rename(columns={'Non-Null':'Non-Null Count'}))

cat_columns = column_df[column_df['Dtype']=='object'].reset_index().Column
non_cat_columns = column_df[column_df['Dtype']!='object'].reset_index().Column

In [34]:
baseline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  Index(['num_subs', 'account_age', 'hasPracticeRecord', 'docPerYear',
       'docOnAusmedPerYear', 'minPerYear', 'minOnAusmedPerYear',
       'learnFromAusmedRatio_num', 'learnFromAusmedRat...
                                                                                              ['canceled',
                                                                                               'incomplete_expired',
                                                                                               'past_due',
                                                                                               'trialing',
                                                                                               'active'],
                                                                                              ['never',
                                                                                               'sometimes',
                                                                                               'usually',
                                                                                               'always'],
                                                                                              ['monthly',
                                                                                               'quarterly',
                                                                                               'annually']],
                                                                                  encoded_missing_value=-1,
                                                                                  handle_unknown='use_encoded_value',
                                                                                  unknown_value=-1)),
                                                                  ('robustscaler',
                                                                   RobustScaler())]),
                                                  ['located', 'Status',
                                                   'access', 'plan_type'])])),
                ('minibatchkmeans', MiniBatchKMeans(n_clusters=4))])

In [81]:
num_features = baseline[0].transformers[0][2]
cat_features = baseline[0].transformers[1][2]

### Loop for k mean calculation

In [100]:
num_features_df = training_data_df_labels[num_features]
num_features_df['col_labels'] = training_data_df_labels['col_labels']

cat_features_df = training_data_df_labels[cat_features]
cat_features_df['col_labels'] = training_data_df_labels['col_labels']

kmean_list = []

for x in num_features:
    kmean_list.append([x,round(num_features_df[x].mean(),1)])
for y in cat_features:
    kmean_list.append([y,cat_features_df[y].mode()[0]])   
    
kmean_list

/var/folders/t8/6sllmnn55m57p9d8z7w2ssrh0000gn/T/ipykernel_76655/2871605007.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_features_df['col_labels'] = training_data_df_labels['col_labels']
/var/folders/t8/6sllmnn55m57p9d8z7w2ssrh0000gn/T/ipykernel_76655/2871605007.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat_features_df['col_labels'] = training_data_df_labels['col_labels']


[['num_subs', 1.1],
 ['account_age', 1452.3],
 ['hasPracticeRecord', 0.9],
 ['docPerYear', 1.6],
 ['docOnAusmedPerYear', 0.9],
 ['minPerYear', 203.8],
 ['minOnAusmedPerYear', 33.7],
 ['learnFromAusmedRatio_num', 0.6],
 ['learnFromAusmedRatio_min', 0.5],
 ['numQueued', 17.1],
 ['numCompletedFromQueue', 0.4],
 ['minQueued', 781.7],
 ['minCompleted', 9.8],
 ['RatioOfCompletion_num', 0.0],
 ['RatioOfCompletion_min', 0.0],
 ['event_cpd_day_diff', 143.8],
 ['doc_in_activation', 3.3],
 ['activated', 0.7],
 ['subscribe_days', 605.7],
 ['GoalsPerYear', 2.0],
 ['ratioOfAchivedGoals', 0.0],
 ['located', 'metropolitan centre'],
 ['Status', 'active'],
 ['access', 'always'],
 ['plan_type', 'quarterly']]